In [2]:
from zumi.zumi import Zumi
from zumi.util.screen import Screen
import IPython.display
from zumi.personality import Personality
import time
from zumi.protocol import Note 
from zumi.util.camera import Camera
from zumi.util.vision import Vision

from zumi.util.color_classifier import ColorClassifier

from zumi.protocol import Note


zumi = Zumi()
screen = Screen()
personality = Personality(zumi, screen)
camera = Camera()
vision = Vision()

def ColorScanner():
    from zumi.util.camera import Camera
    from zumi.util.vision import Vision

    camera = Camera()
    vision = Vision()
    data = None
    try:
        camera.start_camera()
        while data is None:
            image=camera.capture()
            data = vision.find_green_object(image, h_range=7, s_range=60, v_range=40, draw_color=(255, 85, 255),bounding_box=True)
            zumi.play_note(25, 1000)
            zumi.play_note(27, 1000)
            zumi.play_note(29, 1000)
            if data is not None:

                camera.show_image(image)
                camera.clear_output()
    finally:        
        camera.close()
        print("done")
        

def ZumiScanner2():
    camera = Camera()
    vision = Vision()
    camera.start_camera()
    qr_code = None
    try:
        while qr_code == None:
            time_start = time.time()
            image = camera.capture()

            #find the QR code
            qr_code = vision.find_QR_code(image)

            #print if any QR code is detected
            x = vision.get_QR_message(qr_code)

            #this will also clear any print output in jupyter
            camera.clear_output()
    finally:
        camera.close()
        print(x)
        zumi.play_note(25, 1000)
        zumi.play_note(27, 1000)
        zumi.play_note(29, 1000)


    return x




def StopSignScan2():
    data = True
    try:
        camera.start_camera()
        while data == True:
            image=camera.capture()
            data = vision.detect_stop_sign(image)
            if data is True:
                zumi.stop()
                zumi.play_note(25, 1000)
                zumi.play_note(27, 1000)
                zumi.play_note(29, 1000)
            camera.show_image(image)
            camera.clear_output()
    finally:        
        camera.close()
        print("done")
        
        data = False
        
        
        return data



zumi.reset_gyro()
lines = 0
y1 = None
zumi.reset_gyro()
while y1 == None:
    y1 = ZumiScanner2()


    print(y1)
    screen.draw_text_center(y1,font_size=12)

#use turn right or left degree
if y1 == 'factory':
    lines = 4
    zumi.drive_over_markers(road_markers=lines,speed=8,time_out=3.5)
    zumi.turn(-90)
    zumi.forward(speed = 55) # can adjust later
    time.sleep(1)
    zumi.reverse()
    zumi.turn(0)
    while zumi.read_z_angle() > 5:
        zumi.turn_left(1)
    
    while zumi.read_z_angle() < -5:
        zumi.turn_right(1)
    
    
    zumi.drive_over_markers(road_markers=14,speed=8,time_out=5)
    zumi.forward(45,.9)
    zumi.turn(-90) 
# at this point Zumi should have turned to face the funnel at the end of street
    #for i in range(10): # can increase range
    #zumi.go_straight(50, -90)

elif y1 == 'school':
    lines = 8
    zumi.drive_over_markers(road_markers=lines,speed=8,time_out=3.5)
    zumi.turn(90)
    zumi.forward(speed = 55) # can adjust later
    time.sleep(1)
    zumi.reverse()
    zumi.turn(0)
    zumi.reset_gyro()
    while zumi.read_z_angle() > 5:
        zumi.turn_left(1)
    
    while zumi.read_z_angle() < -5:
        zumi.turn_right(1)
    zumi.drive_over_markers(road_markers=14,speed=8,time_out=3.5)
    zumi.forward(45,.9)
    zumi.turn(-90)
    #for i in range(2): # can increase range
        #zumi.go_straight(50, -90)

elif y1 == 'office':
    lines = 12
    zumi.drive_over_markers(road_markers=lines,speed=8,time_out=5.5)
    zumi.turn(-90)
    zumi.forward(speed = 55) # can adjust later
    time.sleep(1)
    zumi.reverse()
    zumi.turn(0)
    while zumi.read_z_angle() > 5:
        zumi.turn_left(1)
    
    while zumi.read_z_angle() < -5:
        zumi.turn_right(1)
    zumi.drive_over_markers(road_markers=14,speed=9,time_out=2)
    zumi.forward(45,.9)
    zumi.turn(-90)
    #for i in range(10): # can increase range
     #   zumi.go_straight(50, -90)

elif y1 == 'museum':
    lines = 16
    zumi.drive_over_markers(road_markers=lines,speed=9,time_out=7)
    zumi.turn(90)
    zumi.forward(speed = 55) # can adjust later
    time.sleep(1)
    zumi.reverse()
    zumi.turn(0)
    while zumi.read_z_angle() > 5:
        zumi.turn_left(1)
    
    while zumi.read_z_angle() < -5:
        zumi.turn_right(1)
    #zumi.drive_over_markers(road_markers=2,speed=20,time_out=10)
    zumi.forward(20,1.5)
    zumi.turn(-90)
    
    
    
    
for i in range(0, 55):
    zumi.go_straight(45,-90)

zumi.stop()
# can adjust parameters later
zumi.funnel_align(speed=2.5, duration=8.5, angle=-90, angle_adj=.5, l_th=90, r_th=90)

# zumi should now at the narrow tip of the funnel facing the stop sign for Section B

y = True
while y:
    y = StopSignScan2()

zumi.forward_avoid_collision(35,2)
zumi.turn_right()
zumi.forward_avoid_collision(35,3)
zumi.turn_right()
zumi.forward_avoid_collision(35,4)
zumi.turn_left()
zumi.forward_avoid_collision(35,4)
zumi.turn_left()
zumi.forward(55,1.5)

zumi.funnel_align(speed=2.5, duration=8.5, angle=-90, angle_adj=.8, l_th=90, r_th=90)

#part c
ColorScanner()
        
        


zumi.forward(20,.5)
zumi.line_follow_gyro_assist(speed=20, duration=10)
zumi.turn_left()
zumi.line_follow_gyro_assist(speed=22, duration=10)
zumi.turn_right()
zumi.line_follow_gyro_assist(speed=22, duration=10)
zumi.turn_right()
f = ZumiScanner2()

if f == 'right':
    
    zumi.line_follow_gyro_assist(speed=20, duration=2.0)
    zumi.turn_right()
    zumi.line_follow_gyro_assist(speed=20, duration=2.0)
    zumi.turn_left()
    zumi.line_follow_gyro_assist(speed=20, duration=7.0)
    zumi.forward(20,1)
    
elif f == 'left':
    zumi.line_follow_gyro_assist(speed=20, duration=2.0)
    zumi.turn_left()
    zumi.line_follow_gyro_assist(speed=20, duration=2.0)
    zumi.turn_right()
    zumi.line_follow_gyro_assist(speed=20, duration=6.0)
    zumi.forward(20,1)    
    
zumi.turn_left(20,.1)
zumi.turn_right(20,.1)
zumi.reverse(10, .4)
zumi.turn_left(20,.1)
zumi.turn_right(20,.1)

screen.draw_text_center(y1 + "            " + f,font_size=12)

Closing PiCamera
left


Starting Zumi 
Pi Zero I2C is available
Verified Pi Zero is the same
Gyroscope previously calibrated
Zumi board detected
OLED Screen detected
Gyroscope & Accelerometer detected
